# Data Wrangling for Predicting Antibiotic Resistance in Gonorrhea

The relevant data is contained in four separate files:
1. A csv file containing all strain samples and minimal inhibitory concentration (MIC) of azithromycin, ciprofloxacin, and ceftrixone
2. A space-separated files containing most common unitigs among resistant samples for each antibiotic:
<ul>
    <li>azithromycin(azm)</li>
    <li>ciprofloxacin(cip)</li>
    <li>ceftrixone(cfx)</li>
</ul>

## 1. Package Importing

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## 2. Data Collection

### 2.1 Data Loading
I began by loading the MIC data, and the three unitig files.

In [2]:
# Loading the MIC data
mic_data = pd.read_csv('../data/external/metadata.csv')
mic_data.head()

,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
0,ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,...,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
1,ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,...,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
2,ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
3,ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,...,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
4,ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,...,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


In [3]:
# Loading azithromycin data
unitigs_azm = pd.read_csv('../data/external/azm_sr_gwas_filtered_unitigs.Rtab', sep='\s', engine='python')
unitigs_azm.head()

,pattern_id,ERR1549286,ERR1549290,ERR1549291,ERR1549287,ERR1549288,ERR1549299,ERR1549292,ERR1549298,ERR1549296,...,ERR2172345,ERR2172346,ERR2172347,ERR2172348,ERR2172349,ERR2172350,ERR2172351,ERR2172352,ERR2172353,ERR2172354
0,CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,TACCGTAACCGGCAATGCGGATATTACGGTC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CAGACGGCATTTTTTTTGCGTTTTTCGGGAGG,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,AACGGGTTTTCAGACGGCATTCGATATCGGGACG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CCAAAAATTACCCGCGTTGACGTAGCTAAAGA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Loading ciprofloxacin data
#unitigs_cip = pd.read_csv('../data/external/cip_sr_gwas_filtered_unitigs.Rtab', sep='\s', engine='python')
#unitigs_cip.head()

In [5]:
# Loading ceftrixone data
#unitigs_cfx = pd.read_csv('../data/external/cfx_sr_gwas_filtered_unitigs.Rtab', sep='\s', engine='python')
#unitigs_cfx.head()

# HEY BRENDAN MAKE SURE YOU DO THIS LATER
### 2.2 Data Joining

Each unitig file needs the samples' MIC data for the respective antibiotic.

In [6]:
# Print shape of each DataFrame
print("mic_data shape: " + str(mic_data.shape))
print("unitigs_azm shape: " + str(unitigs_azm.shape))
#print("unitigs_cip shape: " + str(unitigs_cip.shape))
#print("unitigs_cfx shape: " + str(unitigs_cfx.shape))


mic_data shape: (3786, 31)
unitigs_azm shape: (515, 3972)


'mic_data's rows contain the sample id's while the unitig files have the sample id's as columns, so I will transpose them. Additionally, the 'mic_data' has 3786 rows while the unitigs all have 3972 columns, so some samples might not have MIC data.

In [7]:
# Set index to 'pattern_id' and transpose
unitigs_azm_T = unitigs_azm.set_index('pattern_id').T

# Reset index and rename column 'sample_id'
unitigs_azm_T.reset_index(inplace=True)
unitigs_azm_T = unitigs_azm_T.rename(columns = {'index':'Sample_ID'})
unitigs_azm_T.head()

pattern_id,Sample_ID,CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG,TACCGTAACCGGCAATGCGGATATTACGGTC,CAGACGGCATTTTTTTTGCGTTTTTCGGGAGG,AACGGGTTTTCAGACGGCATTCGATATCGGGACG,CCAAAAATTACCCGCGTTGACGTAGCTAAAGA,CGGACCGGTATTCCGTCGAAATCACCGCCGTCAACCGCCCC,TGAAATTGTCCATCTCGTATGCCGTCTTCTGCTTG,"TACGGTATTGTCCGCATTATTAAACTCAAAACC,AGAAGACGGCATACGAGATGGACAATTTCATCC",GGCATTTTTTTTGCGTTTTTCGGGAGGGGGCGGC,...,ACCGATGAGTTCGCCGGAATCGGTACGATTGAC,CTGCTGGACAAAAAAGGGATTAAAGATATCACC,CGTTCCTTTCGGCGTATTCTCGCCGTTGCGCGGCG,TCACATTTCCGCTTCAGACGGCATCCGATATGA,GAAGCTGCCACGCCGGCAGAAGTGTTGTTTGCGGG,ACGCCGAAAGGAACGTGTATGCTGCCGCCCAACTGCG,ACTCGAATTTTGCAGGATTGGTATCAATGGCGATAATGCGACCGGCTTTGG,"ACCCGGCCCGGGCTGGCAGGCTACGGCTACACCGGTATCC,CACCTTAGGGAATCGTTCCCTTTGGGCCGGG,TACGCCGAAAGGAACGTGTATGCTGCCGCCC,GGGATTGTTGATTGTCGGACTGTTGTGCAACCTC",AGCCTGATTCACCAATGGTTTGTTCATAACAA,TTTTGAGCAGAAAGCAGTCAAAAACAGGGGGATTTTGCCCTTTTGACAGGTTCGAGTGCCG
0,ERR1549286,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,ERR1549290,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,ERR1549291,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3,ERR1549287,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,ERR1549288,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


As I will be merging on 'sample_id', I wanted to compare the sample id's in the unitig data to the MIC data.

In [8]:
# Check if samples in 'mic_data' are in 'unitigs_azm_T'
print('samples from mic_data that are in unitigs_azm_t:')
print(mic_data['Sample_ID'].isin(unitigs_azm_T['Sample_ID']).value_counts())
# Check samples in 'unitig_azm_T' are in 'mic_data'
print('samples from unitigs_azm_ that are in mic_data:')
print(unitigs_azm_T['Sample_ID'].isin(mic_data['Sample_ID']).value_counts())

samples from mic_data that are in unitigs_azm_t:
True    3786
Name: Sample_ID, dtype: int64
samples from unitigs_azm_ that are in mic_data:
True     3786
False     185
Name: Sample_ID, dtype: int64


All of the samples from 'mic_data' are in 'unitigs_azm_T' but 185 samples are not in 'mic_data'

In [9]:
# Joining with 'mic_data' column 'Azithromycin' on 'sample_id' and 'Sample_ID'
unitigs_azm = unitigs_azm_T.merge(mic_data[['Sample_ID','Year','Country','Continent','Azithromycin']], 
                                how = 'left', on = 'Sample_ID')
unitigs_azm = unitigs_azm.set_index('Sample_ID')
unitigs_azm.head()

,CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG,TACCGTAACCGGCAATGCGGATATTACGGTC,CAGACGGCATTTTTTTTGCGTTTTTCGGGAGG,AACGGGTTTTCAGACGGCATTCGATATCGGGACG,CCAAAAATTACCCGCGTTGACGTAGCTAAAGA,CGGACCGGTATTCCGTCGAAATCACCGCCGTCAACCGCCCC,TGAAATTGTCCATCTCGTATGCCGTCTTCTGCTTG,"TACGGTATTGTCCGCATTATTAAACTCAAAACC,AGAAGACGGCATACGAGATGGACAATTTCATCC",GGCATTTTTTTTGCGTTTTTCGGGAGGGGGCGGC,TATATAAGGGGTTGCCGTTCCGCAGTTGGGCGGCAGCATAC,...,GAAGCTGCCACGCCGGCAGAAGTGTTGTTTGCGGG,ACGCCGAAAGGAACGTGTATGCTGCCGCCCAACTGCG,ACTCGAATTTTGCAGGATTGGTATCAATGGCGATAATGCGACCGGCTTTGG,"ACCCGGCCCGGGCTGGCAGGCTACGGCTACACCGGTATCC,CACCTTAGGGAATCGTTCCCTTTGGGCCGGG,TACGCCGAAAGGAACGTGTATGCTGCCGCCC,GGGATTGTTGATTGTCGGACTGTTGTGCAACCTC",AGCCTGATTCACCAATGGTTTGTTCATAACAA,TTTTGAGCAGAAAGCAGTCAAAAACAGGGGGATTTTGCCCTTTTGACAGGTTCGAGTGCCG,Year,Country,Continent,Azithromycin
Sample_ID,,,,,,,,,,,,,,,,,,,,,
ERR1549286,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2015.0,UK,Europe,>256
ERR1549290,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2015.0,UK,Europe,>256
ERR1549291,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2015.0,UK,Europe,>256
ERR1549287,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2015.0,UK,Europe,>256
ERR1549288,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,2015.0,UK,Europe,>256


## 3. Data Definition

### 3.1 Renaming Columns

I renamed the 'Azithromycin' column to be more accurately 'mic_azm' as it is the MIC to azithromycin. The remaining columns are descriptive for what they contain.

In [10]:
# Rename 'Azithromycin' column
unitigs_azm = unitigs_azm.rename(columns = {'Azithromycin':'mic_azm'})

### 3.2 Checking Data Types

In [11]:
# Checking info of unitigs_azm
print(unitigs_azm.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3971 entries, ERR1549286 to ERR2172354
Columns: 519 entries, CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG to mic_azm
dtypes: float64(1), int64(515), object(3)
memory usage: 15.8+ MB
None


There is one float, 515 ints, and 3 objects. I suspect these are 'Year', the unitigs, and 'Country', 'Continent', 'mic_azm', respectively.

In [12]:
# Find key of column that is a float
unitigs_azm.select_dtypes(float).keys()

Index(['Year'], dtype='object')

The float is 'Year'.

In [13]:
# Define function to count objects
def count_obj(df,col):
    """For the designated column col in the DataFrame df,
        produce a value count as a count and as a percentage"""
    count = pd.concat([unitigs_azm[col].value_counts(),
            df[col].value_counts()/df[col].notnull().sum()*100], axis = 1)
    count.columns = ['count','%']
    return count

In [14]:
# Print description and value counts for year
print(unitigs_azm['Year'].describe())
print(count_obj(unitigs_azm,'Year'))

count    3536.000000
mean     2010.300622
std         4.670817
min      1979.000000
25%      2009.000000
50%      2012.000000
75%      2013.000000
max      2017.000000
Name: Year, dtype: float64
        count          %
2013.0   1196  33.823529
2015.0    439  12.415158
2010.0    284   8.031674
2011.0    275   7.777149
2012.0    222   6.278281
2009.0    219   6.193439
2004.0    126   3.563348
2007.0    113   3.195701
1998.0     99   2.799774
2005.0     82   2.319005
2008.0     78   2.205882
2002.0     59   1.668552
2006.0     53   1.498869
2000.0     51   1.442308
2014.0     49   1.385747
2001.0     47   1.329186
2003.0     40   1.131222
2016.0     32   0.904977
1999.0     29   0.820136
1997.0     11   0.311086
2017.0      8   0.226244
1989.0      5   0.141403
1993.0      5   0.141403
1990.0      3   0.084842
1991.0      3   0.084842
1996.0      3   0.084842
1992.0      2   0.056561
1986.0      1   0.028281
1979.0      1   0.028281
1994.0      1   0.028281


The samples are from 1979 to 2017 with most from 2009 to 2013. 2013 was the most common year with 30% of the samples. This makes the samples fairly recent enough to year responsible results.

In [15]:
# Find keys of columns that are objects
unitigs_azm.select_dtypes(object).keys()

Index(['Country', 'Continent', 'mic_azm'], dtype='object')

In [16]:
# Print counts for 'Continent' and 'Country'
print('Continent')
print(count_obj(unitigs_azm,'Continent'))
print('Country')
print(count_obj(unitigs_azm,'Country'))

Continent
         count          %
Europe    1675  44.253633
America   1505  39.762219
Oceania    411  10.858653
Asia       150   3.963012
Africa      44   1.162483
Country
             count          %
USA           1121  29.616909
UK             493  13.025099
New_Zealand    400  10.568032
Canada         372   9.828269
Spain          132   3.487450
...            ...        ...
Chile            1   0.026420
Ecuador          1   0.026420
Caribbean        1   0.026420
Romania          1   0.026420
Lithuania        1   0.026420

[65 rows x 2 columns]


Most samples are from Europe and America, with 29.6% from the USA and 13.0% from the UK, indicating that a model generated by this data may be most accurate in these regions.

In [17]:
# Find count of 'mic_azm'
count_obj(unitigs_azm,'mic_azm')

,count,%
0.25,698,20.057471
0.5,697,20.028736
0.125,297,8.534483
1,277,7.959770
2,186,5.344828
8,151,4.339080
0.12,110,3.160920
0.06,107,3.074713
>256,101,2.902299
0.19,97,2.787356


The 'mic_azm' column mostly contains floats but there are 5 distinct floats: 4 where the sample was not inhibited by the maximum concentration of azm used in its study (>256, >=512, >=256/>=512, >=64/>=512) and 1 where the sample was inhibited by the minimum concentration used (<=0.008).

This poses two issues:
1. Before getting more accurate summary statistics on theses, I needed to replace these with floats and convert the datatype of the column. 
2. MIC of >=256/>=512 or >=64/>=512 provides a very large range of possible MICs.

The first issue can be solved by replacing said values with the most accurate MIC. As MIC is determined by dilution, our safest assumption that twice the maximum concentration is the most accurate MIC, so 512 for >256 and 1024 for >=512. It is important to retain these values as they are the most resistant to the antibiotic. Similarly, for <=0.008, we will assume an MIC of one dilution further: 0.004, which is still far below most values and well within the sensitive range for the antibiotic.

For the second issue, these values reflect 2 samples and can therefore be replaced with None values given the size of our data set.

In [23]:
# Set replacement dictionary
mic_replace = {'>=256/>=512':None,
               '>=64/>=512':None,
               '>256':512,
               '>=512':1024,
               '<=0.008':0.004}

# Replace non-float 'mic_azm' values
unitigs_azm['mic_azm'] = unitigs_azm['mic_azm'].replace(mic_replace)

In [24]:
count_obj(unitigs_azm,'mic_azm')

,count,%
0.25,698,20.069005
0.5,697,20.040253
0.125,297,8.539390
1,277,7.964347
2,186,5.347901
8,151,4.341576
0.12,110,3.162737
0.06,107,3.076481
512,101,2.903968
0.19,97,2.788959


Now all of 'mic_azm' is a float. I can change its datatype form object to float and get descriptive summary statistics.

In [26]:
# Conver 'mic_azm' to float
unitigs_azm['mic_azm'] = unitigs_azm['mic_azm'].astype(float)
unitigs_azm['mic_azm'].describe()

count    3478.000000
mean       17.118781
std        90.864520
min         0.004000
25%         0.125000
50%         0.380000
75%         1.000000
max      1024.000000
Name: mic_azm, dtype: float64

The range of MIC is 0.004 to 1024, which is expectedly two values I set due to being outside the minimum/maximum range of concentrations tested. The median MIC is 0.38,

In [22]:
# Find keys of columns that are ints
unitigs_azm.select_dtypes(int).keys()

Index(['CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG',
       'TACCGTAACCGGCAATGCGGATATTACGGTC', 'CAGACGGCATTTTTTTTGCGTTTTTCGGGAGG',
       'AACGGGTTTTCAGACGGCATTCGATATCGGGACG',
       'CCAAAAATTACCCGCGTTGACGTAGCTAAAGA',
       'CGGACCGGTATTCCGTCGAAATCACCGCCGTCAACCGCCCC',
       'TGAAATTGTCCATCTCGTATGCCGTCTTCTGCTTG',
       'TACGGTATTGTCCGCATTATTAAACTCAAAACC,AGAAGACGGCATACGAGATGGACAATTTCATCC',
       'GGCATTTTTTTTGCGTTTTTCGGGAGGGGGCGGC',
       'TATATAAGGGGTTGCCGTTCCGCAGTTGGGCGGCAGCATAC',
       ...
       'ACCGATGAGTTCGCCGGAATCGGTACGATTGAC',
       'CTGCTGGACAAAAAAGGGATTAAAGATATCACC',
       'CGTTCCTTTCGGCGTATTCTCGCCGTTGCGCGGCG',
       'TCACATTTCCGCTTCAGACGGCATCCGATATGA',
       'GAAGCTGCCACGCCGGCAGAAGTGTTGTTTGCGGG',
       'ACGCCGAAAGGAACGTGTATGCTGCCGCCCAACTGCG',
       'ACTCGAATTTTGCAGGATTGGTATCAATGGCGATAATGCGACCGGCTTTGG',
       'ACCCGGCCCGGGCTGGCAGGCTACGGCTACACCGGTATCC,CACCTTAGGGAATCGTTCCCTTTGGGCCGGG,TACGCCGAAAGGAACGTGTATGCTGCCGCCC,GGGATTGTTGATTGTCGGACTGTTGTGCAACCTC',
       '